In [1]:
!pip install phe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from phe import paillier
import time
import psutil
import os
from threading import Thread, Lock

In [3]:
X_train = pd.read_csv('X_train_classification.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train_classification.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test_classification.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test_classification.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

n_samples,n_attributes=X_train.shape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
bias = 0
lr = 0.01
C = 0.01 # change C=10 or 1 if high attributes are involved

In [4]:
# Paillier key generation
public_key, private_key = paillier.generate_paillier_keypair()

In [5]:
# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [6]:
# Function to handle encrypted multiplication at the edge device
def paillier_multiplication_edge(a, b):
    with monitor_lock:  # Acquire lock to exclude this part from CPU and RAM monitoring
        a = private_key.decrypt(a)
        a=round(a,6)
        a = a * b
    return a

In [7]:
# Function to handle encrypted multiplication at the cloud
def paillier_multiplication_cloud(a, b):
    noise1 = 0.01
    noise2 = 0.02
    a = a + noise1
    sub_mass = (b * noise1) * (-1)
    sol = paillier_multiplication_edge(a, b)
    sol = sol + sub_mass
    return sol

In [8]:
# enc_X_train = [[public_key.encrypt(j) for j in i] for i in X_train]
# enc_y_train = [public_key.encrypt(int(i)) for i in y_train]
enc_X_train=[]
count=0
for i in X_train:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_train.append(row_in)
  count+=1

enc_y_train = [public_key.encrypt(int(i)) for i in y_train]

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted
row 20 encrypted
row 21 encrypted
row 22 encrypted
row 23 encrypted
row 24 encrypted
row 25 encrypted
row 26 encrypted
row 27 encrypted
row 28 encrypted
row 29 encrypted
row 30 encrypted
row 31 encrypted
row 32 encrypted
row 33 encrypted
row 34 encrypted
row 35 encrypted
row 36 encrypted
row 37 encrypted
row 38 encrypted
row 39 encrypted
row 40 encrypted
row 41 encrypted
row 42 encrypted
row 43 encrypted
row 44 encrypted
row 45 encrypted
row 46 encrypted
row 47 encrypted
row 48 encrypted
row 49 encrypted
row 50 encrypted
row 51 encrypted
row 52 encrypted
row 53 encrypted
row 54 encrypted
row 55 encrypted
row 56 encrypted
row 57 encrypted
row 58 encrypted
row 59 

In [9]:
len(enc_X_train)

80

In [10]:
def paillier_scalar_rescaling(r):
  r=private_key.decrypt(r)
  r=round(r,9)
  r=public_key.encrypt(r)
  return r

In [11]:
# margin calculation:
weights=[]
for i in range(X_train.shape[1]):
  weights.append(public_key.encrypt(0))

weights

In [12]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [13]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 450

In [14]:
time.sleep(15)

In [15]:
results={}
def svm_paillier(enc_X_train,enc_y_train,weights,bias,lr,C):
  train_time_start=time.time()
  for _ in range(3):
    for w in range(n_samples):
        xi=enc_X_train[w]
        yi=enc_y_train[w]

        mm=0
        summ=0
        for i,j in zip(weights,xi):
          r=None
          r=paillier_multiplication_cloud(j,i)
          mm=summ+r

        # mm=paillier_scalar_rescaling(mm)
        l2=mm+bias


        lot=paillier_multiplication_cloud(yi,l2)
        # margins=1-summ

        if private_key.decrypt(lot) < 1:
          s=C*yi
          m2=[]
          for m in xi:
            m2.append(paillier_multiplication_cloud(m,s))

          r3=[]
          for i,j in zip(weights,m2):
            r3.append(i-j)

          r4=[]
          for i in r3:
            r4.append(lr*i)

          r5=[]
          for i,j in zip(weights,r4):
            r5.append(i-j)

          weights=r5
          bias=bias-lr*(-C*yi)

        else:
          r6=[]
          for i in weights:
            m=i*lr
            r6.append(i-m)

          weights=r6

        print(f"epoch = {_} iter {w} executed")


        if w%25==0:
          weights2=[]
          for i in weights:
            weights2.append(paillier_scalar_rescaling(i))
          weights=weights2
          bias=paillier_scalar_rescaling(bias)

  results['weights']=weights
  results['bias']=bias
  train_time_end=time.time()
  results['train_time']=train_time_end-train_time_start
  print(results['train_time'])

In [16]:
train_thread = Thread(target=svm_paillier, args=(enc_X_train, enc_y_train, weights, bias, lr, C))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 49.3% | RAM utilization: 1.5687041682530176%
CPU utilization: 63.7% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 0 executed
CPU utilization: 69.8% | RAM utilization: 1.5687041682530176%
CPU utilization: 92.1% | RAM utilization: 1.5687041682530176%
CPU utilization: 102.0% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 1 executed
CPU utilization: 98.2% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 2 executed
CPU utilization: 95.7% | RAM utilization: 1.5687041682530176%
CPU utilization: 96.6% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 3 executed
CPU utilization: 99.8% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 4 executed
CPU utilization: 99.5% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 5 executed
CPU utilization: 97.8% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 6 executed
CPU utilization: 107.5% | RAM utilization: 1.5687041682530176%
epoch = 0 iter 7 executed
CPU utilization: 100.1% | RAM utilization: 1.5

In [17]:
results_unenc=[]
for i in results['weights']:
  t=private_key.decrypt(i)
  results_unenc.append(t)

In [18]:
results_unenc

[0.00020519373071707902, 0.00715797659311718]

In [19]:
private_key.decrypt(results['bias'])

3.877106968808164e-20

In [20]:
enc_X_test=[]
count=0
for i in X_test:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_test.append(row_in)
  count+=1

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted


In [21]:
def return_sign(x):
  x1=private_key.decrypt(x)
  return np.sign(x1)

In [22]:
def test_svm(enc_X_test,weights,bias):
  y_pred=[]
  count2=0
  for xi in enc_X_test:
    p=0
    for i,j in zip(xi,weights):
      r=paillier_multiplication_cloud(i,j)
      p=p+r
    p=p+bias
    y_pred.append(return_sign(p))
    print(f"item {count2} tested")
    count2+=1

  return y_pred

In [23]:
test_time_start=time.time()
y_pred=test_svm(enc_X_test,results['weights'],results['bias'])
test_time_end=time.time()
results['test_time']=test_time_end-test_time_start

item 0 tested
item 1 tested
item 2 tested
item 3 tested
item 4 tested
item 5 tested
item 6 tested
item 7 tested
item 8 tested
item 9 tested
item 10 tested
item 11 tested
item 12 tested
item 13 tested
item 14 tested
item 15 tested
item 16 tested
item 17 tested
item 18 tested
item 19 tested


In [24]:
print(results['test_time'])

9.77422308921814


In [25]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.85

In [26]:
import joblib
joblib.dump((results_unenc, private_key.decrypt(results['bias']), accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_paillier.pkl')

['variables_paillier.pkl']